In [1]:
import os
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np

In [2]:
df = pd.read_csv('total.csv')
df

,Unnamed: 0,Date,Issuer,Symbol,Lead/Joint-Lead _Managers,Offer_Price,Opening_Price,1st Day_Close,1st Day_Chng,underpriced,Year,onemonth,avg_trend,Volume,Industry,text
0,0,2020-01-17,I-Mab,IMAB,Jefferies/ CICC,14.0,14.75,12.75,-0.0893,0,2020,2019-12-17,16.31250,536.3395,Biotechnology,the following summary is qualified in its enti...
1,1,2020-01-17,LIZHI,LIZI,Credit Suisse/ Citigroup,11.0,11.03,11.63,0.0573,1,2020,2019-12-17,18.37500,536.3395,Internet Content & Information,NaN
2,2,2020-01-17,Phoenix Tree Holdings Ltd.,DNK,Citigroup/ Credit Suisse/ J.P. Morgan,13.5,13.50,13.50,0.0000,0,2020,2019-12-17,NaN,536.3395,NaN,this summary highlights selected information c...
3,3,2020-01-17,Velocity Financial,VEL,Wells Fargo Securities/ Citigroup/ JMP Securities,13.0,13.85,13.51,0.0392,1,2020,2019-12-17,19.59375,536.3395,Mortgage Finance,NaN
4,4,2020-01-24,Gores Holdings IV,GHIVU,Deutsche Bank Securities,10.0,10.52,10.45,0.0450,1,2020,2019-12-24,6.25000,536.3395,Shell Companies,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3011,3011,2004-12-15,Advance America Cash Advance,AEA,Morgan Stanley,15.0,20.50,20.50,0.3667,1,2004,2004-11-14,9.28125,431.3597,NaN,NaN
3012,3012,2004-12-15,Arbinet-thexchange,ARBX,Merrill Lynch/Lehman Brothers,17.5,26.85,29.00,0.6571,1,2004,2004-11-14,NaN,431.3597,NaN,NaN
3013,3013,2004-12-15,KMG AMERICA,KMA,Friedman Billings Ramsey,9.5,10.50,10.80,0.1368,1,2004,2004-11-14,NaN,431.3597,NaN,NaN
3014,3014,2004-12-16,Warren Resources,WRES,KeyBanc Capital Markets,7.5,10.00,9.20,0.2267,1,2004,2004-11-15,6.68750,431.3597,NaN,NaN


In [5]:
df = df.dropna(subset=['text'])
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

/Users/cleverpan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,Issuer,Symbol,Lead/Joint-Lead _Managers,Offer_Price,Opening_Price,1st Day_Close,1st Day_Chng,underpriced,Year,onemonth,avg_trend,Volume,Industry,text
0,2020-01-17,I-Mab,IMAB,Jefferies/ CICC,14.0,14.75,12.75,-0.0893,0,2020,2019-12-17,16.31250,536.3395,Biotechnology,the following summary is qualified in its enti...
2,2020-01-17,Phoenix Tree Holdings Ltd.,DNK,Citigroup/ Credit Suisse/ J.P. Morgan,13.5,13.50,13.50,0.0000,0,2020,2019-12-17,NaN,536.3395,NaN,this summary highlights selected information c...
6,2020-01-28,AnPac Bio-Medical Science,ANPC,WestPark Capital,12.0,12.00,11.25,-0.0625,0,2020,2019-12-28,3.12500,536.3395,Biotechnology,the following summary highlights selected info...
7,2020-01-29,Annovis Bio,ANVS,ThinkEquity (a division of Fordham Financial M...,6.0,6.63,9.59,0.5983,1,2020,2019-12-29,14.71875,536.3395,Biotechnology,this summary highlights information contained ...
10,2020-01-31,Arcutis Biotherapeutics,ARQT,Goldman Sachs/ Cowen/ Guggenheim Securities,17.0,23.05,21.80,0.2824,1,2020,2019-12-31,21.56250,536.3395,Biotechnology,this summary highlights selected information c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2992,2004-12-09,Bill Barrett,BBG,Goldman Sachs,25.0,27.50,29.05,0.1620,1,2004,2004-11-08,16.65625,431.3597,NaN,this summary contains basic information about ...
2997,2004-12-13,Comstock Homebuilding Companies,CHCI,BB&T Capital Markets,16.0,19.00,16.75,0.0469,1,2004,2004-11-12,NaN,431.3597,NaN,summary historical and pro forma financial and...
2999,2004-12-13,BlueLinx Holdings,BXC,Goldman Sachs/Morgan Stanley,13.5,13.50,13.50,0.0000,0,2004,2004-11-12,5.96875,431.3597,NaN,cautionary statement concerning forward lookin...
3003,2004-12-14,Las Vegas Sands,LVS,Goldman Sachs,29.0,41.90,46.56,0.6055,1,2004,2004-11-13,18.87500,431.3597,Resorts & Casinos,disclosure regarding forward looking statement...


In [6]:
with open('stopwords.txt' ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []

In [17]:
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

def wordcount(text):
    return len(tokenizer(text))

In [8]:
# Loading positive words
with open('positivewords.txt','r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')

In [9]:
# Loading negative words
with open('negativewords.txt' ,'r') as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

In [10]:
# Loading uncertain words
with open('uncertainwords.txt' ,'r') as uncerfile:
    uncertainword=uncerfile.read().lower()
uncertainWordList=uncertainword.split('\n')

In [11]:
# Loading weak modal words
with open('weakmodal.txt' ,'r') as weakfile:
    weakword=weakfile.read().lower()
weakWordList=weakword.split('\n')

In [12]:
# Loading negative words
with open('strongmodal.txt' ,'r') as strongfile:
    strongword=strongfile.read().lower()
strongWordList=strongword.split('\n')

In [13]:
# Calculating proportion 
def proport(text, wordlist):
    numWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in wordlist:
            numWords += 1
    return numWords / len(rawToken)

In [14]:
def positive_pro(text):
    return proport(text, positiveWordList)

def negative_pro(text):
    return proport(text, negativeWordList)

def uncertain_pro(text):
    return proport(text, uncertainWordList)

def weak_pro(text):
    return proport(text, weakWordList)

def strong_pro(text):
    return proport(text, strongWordList)

In [15]:
df['Positive'] = df['text'].apply(positive_pro)

/Users/cleverpan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df['Negative'] = df['text'].apply(negative_pro)
df['Uncertain'] = df['text'].apply(uncertain_pro)
df['Weak_Modal'] = df['text'].apply(weak_pro)
df['Strong_Modal'] = df['text'].apply(strong_pro)

/Users/cleverpan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/cleverpan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/cleverpan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [18]:
df

,Date,Issuer,Symbol,Lead/Joint-Lead _Managers,Offer_Price,Opening_Price,1st Day_Close,1st Day_Chng,underpriced,Year,onemonth,avg_trend,Volume,Industry,text,Positive,Negative,Uncertain,Weak_Modal,Strong_Modal
0,2020-01-17,I-Mab,IMAB,Jefferies/ CICC,14.0,14.75,12.75,-0.0893,0,2020,2019-12-17,16.31250,536.3395,Biotechnology,the following summary is qualified in its enti...,0.014345,0.025301,0.020121,0.012296,0.006323
2,2020-01-17,Phoenix Tree Holdings Ltd.,DNK,Citigroup/ Credit Suisse/ J.P. Morgan,13.5,13.50,13.50,0.0000,0,2020,2019-12-17,NaN,536.3395,NaN,this summary highlights selected information c...,0.012495,0.024447,0.018521,0.012373,0.006822
6,2020-01-28,AnPac Bio-Medical Science,ANPC,WestPark Capital,12.0,12.00,11.25,-0.0625,0,2020,2019-12-28,3.12500,536.3395,Biotechnology,the following summary highlights selected info...,0.009449,0.026070,0.021779,0.013393,0.006546
7,2020-01-29,Annovis Bio,ANVS,ThinkEquity (a division of Fordham Financial M...,6.0,6.63,9.59,0.5983,1,2020,2019-12-29,14.71875,536.3395,Biotechnology,this summary highlights information contained ...,0.014744,0.031588,0.025928,0.017201,0.007694
10,2020-01-31,Arcutis Biotherapeutics,ARQT,Goldman Sachs/ Cowen/ Guggenheim Securities,17.0,23.05,21.80,0.2824,1,2020,2019-12-31,21.56250,536.3395,Biotechnology,this summary highlights selected information c...,0.013447,0.029826,0.023794,0.014746,0.007510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2992,2004-12-09,Bill Barrett,BBG,Goldman Sachs,25.0,27.50,29.05,0.1620,1,2004,2004-11-08,16.65625,431.3597,NaN,this summary contains basic information about ...,0.006014,0.016253,0.016870,0.007653,0.006646
2997,2004-12-13,Comstock Homebuilding Companies,CHCI,BB&T Capital Markets,16.0,19.00,16.75,0.0469,1,2004,2004-11-12,NaN,431.3597,NaN,summary historical and pro forma financial and...,0.000000,0.002770,0.005540,0.000000,0.000000
2999,2004-12-13,BlueLinx Holdings,BXC,Goldman Sachs/Morgan Stanley,13.5,13.50,13.50,0.0000,0,2004,2004-11-12,5.96875,431.3597,NaN,cautionary statement concerning forward lookin...,0.001931,0.009653,0.013514,0.009653,0.000000
3003,2004-12-14,Las Vegas Sands,LVS,Goldman Sachs,29.0,41.90,46.56,0.6055,1,2004,2004-11-13,18.87500,431.3597,Resorts & Casinos,disclosure regarding forward looking statement...,0.000000,0.002119,0.002119,0.002119,0.000000


In [19]:
df['wordcount'] = df['text'].apply(wordcount)

/Users/cleverpan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df.head()

,Date,Issuer,Symbol,Lead/Joint-Lead _Managers,Offer_Price,Opening_Price,1st Day_Close,1st Day_Chng,underpriced,Year,...,avg_trend,Volume,Industry,text,Positive,Negative,Uncertain,Weak_Modal,Strong_Modal,wordcount
0,2020-01-17,I-Mab,IMAB,Jefferies/ CICC,14.0,14.75,12.75,-0.0893,0,2020,...,16.31250,536.3395,Biotechnology,the following summary is qualified in its enti...,0.014345,0.025301,0.020121,0.012296,0.006323,142487
2,2020-01-17,Phoenix Tree Holdings Ltd.,DNK,Citigroup/ Credit Suisse/ J.P. Morgan,13.5,13.50,13.50,0.0000,0,2020,...,NaN,536.3395,NaN,this summary highlights selected information c...,0.012495,0.024447,0.018521,0.012373,0.006822,90439
6,2020-01-28,AnPac Bio-Medical Science,ANPC,WestPark Capital,12.0,12.00,11.25,-0.0625,0,2020,...,3.12500,536.3395,Biotechnology,the following summary highlights selected info...,0.009449,0.026070,0.021779,0.013393,0.006546,86461
7,2020-01-29,Annovis Bio,ANVS,ThinkEquity (a division of Fordham Financial M...,6.0,6.63,9.59,0.5983,1,2020,...,14.71875,536.3395,Biotechnology,this summary highlights information contained ...,0.014744,0.031588,0.025928,0.017201,0.007694,59009
10,2020-01-31,Arcutis Biotherapeutics,ARQT,Goldman Sachs/ Cowen/ Guggenheim Securities,17.0,23.05,21.80,0.2824,1,2020,...,21.56250,536.3395,Biotechnology,this summary highlights selected information c...,0.013447,0.029826,0.023794,0.014746,0.007510,83887


In [25]:
df.to_csv('sentiment.csv')

OSError: [Errno 28] No space left on device